In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('station_hour.csv')

C:\Users\Naveen\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df = df.dropna()

In [4]:
df=df[['PM2.5','PM10','NO2','NOx','CO','AQI']]

In [5]:
df = df.head(50000)

In [6]:
df.shape

(50000, 6)

In [7]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [9]:
import numpy as np
mu = np.mean(X_train, 0)
sigma = np.std(X_train, 0)

X_train = (X_train - mu ) / sigma

#We use the same mean and SD as the one of X_train as we dont know the mean of X_test
X_test = (X_test - mu ) / sigma

#Standardizing the y_train data
mu_y = np.mean(y_train, 0)
sigma_y = np.std(y_train, 0, ddof = 0)

y_train = (y_train - mu_y ) / sigma_y

In [10]:
y_train = y_train.reshape(len(y_train),1)
y_test = y_test.reshape(len(y_test),1)
y_pred = np.zeros(y_test.shape)
y_train.shape, y_test.shape,y_pred.shape

((40000, 1), (10000, 1), (10000, 1))

In [17]:
n_neigh = 28

for row in range(len(X_test)):
    weighted_points = np.zeros(n_neigh)
    sum1 = 0
    euclidian_distance = np.sqrt(np.sum((X_train - X_test[row])**2, axis = 1 ))
    sorted_distance = euclidian_distance[np.argsort(euclidian_distance, axis = 0)[:n_neigh]]
    closest_points = y_train[np.argsort(euclidian_distance, axis = 0)[:n_neigh]]
    j = 0
    for i in range(n_neigh,0,-1):
        weighted_points[j] = closest_points[j]*i
        j += 1
        sum1 += i
    
    y_pred[row] = sum(weighted_points)/sum1* sigma_y + mu_y

In [18]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.7589408606062822

In [19]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

1931.6176554484457